In [2]:
import apache_beam as beam
from apache_beam.runners import DataflowRunner
from apache_beam.options.pipeline_options import PipelineOptions
import apache_beam.transforms.window as window
from apache_beam.metrics import Metrics

# B. Apache Beam ML Libraries
from apache_beam.ml.inference.base import ModelHandler
from apache_beam.ml.inference.base import RunInference
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

# C. Python Libraries
from datetime import datetime
import argparse
import logging
import json

# Import libraries
from google.cloud import pubsub_v1
from google.pubsub_v1.types import Encoding
import logging
import json

beam.options.pipeline_options.PipelineOptions.allow_non_parallel_instruction_output = True
DataflowRunner.__test__ = False


In [7]:
project_id = "data-project-2-449815"
subscription_ayuda = "ayuda-sub"
subscription_voluntarios = "voluntarios-sub"

In [1]:
def DecodificarMensaje(msg):

    message = msg.decode('utf-8')

    decodificado = json.loads(message)

    return decodificado["Necesidad"], decodificado

In [20]:
import sys
import pandas as pd

sys.argv = sys.argv[:1]



def run():
    with beam.Pipeline(options=PipelineOptions(streaming=True, save_main_session=True)) as p:

        datos_ayuda = (
            p 
            | "Leer datos de ayuda" >> beam.io.ReadFromPubSub(subscription=f'projects/{project_id}/subscriptions/{subscription_ayuda}')
            | "Decode msg" >> beam.Map(DecodificarMensaje) 
            | "Fixed Window para los datos de Ayuda" >> beam.WindowInto(beam.window.FixedWindows(60))
            | "PrintMessage" >> beam.Map(print) 
        )
    
        datos_voluntarios = (
            p 
            | "Leer datos de Voluntarios" >> beam.io.ReadFromPubSub(subscription=f'projects/{project_id}/subscriptions/{subscription_voluntarios}')
            | "Decodificar mensage" >> beam.Map(DecodificarMensaje)
            | "Fixed Window para los datos de los voluntarios" >> beam.WindowInto(beam.window.FixedWindows(60))
            | "PrintMessage" >> beam.Map(print)
        )

        grouped_data = (
            datos_ayuda, datos_voluntarios) | "Juntar por Necesidad"  >> beam.CoGroupByKey()
                                            
        leer_mensajes = (
            grouped_data
            | "PrintMessage" >> beam.Map(print) 
        )
    

# Run Process
logging.info("The process started")
run()

RuntimeError: A transform with label "PrintMessage" already exists in the pipeline. To apply a transform with a specified label, write pvalue | "label" >> transform or use the option "auto_unique_labels" to automatically generate unique transform labels. Note "auto_unique_labels" could cause data loss when updating a pipeline or reloading the job state. This is not recommended for streaming jobs.

('Hogar', {'ID': 'A96369', 'Nombre': 'Artemio Matas Simo', 'Edad': 79, 'Telefono': '653717184', 'Nivel_Urgencia': 'Media', 'Necesidad': 'Hogar', 'Timestamp': '2025-02-06T00:08:54.532333', 'Ubicacion': {'latitud': 39.470601, 'longitud': -0.360546}})
('Herramientas manuales', {'ID': 'V1019', 'Nombre': 'Charo Maria Dolores Pazos Martinez', 'Edad': 52, 'Telefono': '638611240', 'Necesidad': 'Herramientas manuales', 'Nivel de Urgencias': 'Mañana', 'Timestamp': '2025-02-06T00:08:54.678724', 'Ubicacion': {'latitud': 39.477784, 'longitud': -0.356863}})
('Comida', {'ID': 'A4064', 'Nombre': 'Gertrudis del Ferrer', 'Edad': 29, 'Telefono': '630849265', 'Nivel_Urgencia': 'Alta', 'Necesidad': 'Comida', 'Timestamp': '2025-02-06T00:08:57.525078', 'Ubicacion': {'latitud': 39.48619, 'longitud': -0.393083}})
('Comida', {'ID': 'V91662', 'Nombre': 'Flora de Canizares', 'Edad': 21, 'Telefono': '631097292', 'Necesidad': 'Comida', 'Nivel de Urgencias': 'Mañana', 'Timestamp': '2025-02-06T00:08:58.046505', 'Ubic